In [1]:
import numpy as np
import re
from itertools import combinations

with open("input/day20_input.txt", "r") as f:
    data = [l.rstrip().replace(".", "0").replace("#", "1") for l in f.readlines()]

In [2]:
tile_number = None
tile_data = []
tiles = {}

for line in data:
    if line.startswith("Tile"):
        tile_number = int(re.findall(r"\d+", line)[0])
    elif line.isnumeric():
        tile_data.append([int(l) for l in line])
    else:
        tiles[tile_number] = np.array(tile_data)
        tile_number = None
        tile_data = []
        
tiles[tile_number] = np.array(tile_data)

In [3]:
def compare_tiles(x1, x2):
    edges = [(0, 0), (0, -1), (-1, 0), (-1, -1)]
    flips = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
    
    tests = []
    for fa, fb in flips:
        for a, b in edges:
            tests.append(np.array_equal(x1[a][::fa], x2[b][::fb]))
            tests.append(np.array_equal(x1[:, a][::fa], x2[:, b][::fb]))
            tests.append(np.array_equal(x1[a][::fa], x2[:, b][::fb]))
            tests.append(np.array_equal(x1[:, a][::fa], x2[b][::fb]))
    return np.any(tests)

In [4]:
# Find corner tiles
matching_edges = {}

for k1 in tiles.keys():
    count = 0
    for k2 in tiles.keys():
        if k1 == k2:
            continue
        else:
            if compare_tiles(tiles[k1], tiles[k2]):
                count += 1
                
    matching_edges[k1] = count

corner_ids = [k for k, v in matching_edges.items() if v == 2]
np.product(corner_ids, dtype=np.int64)

13983397496713

# Part 2

In [5]:
def find_piece(index, puzzle, exclude_list=[]):
    r, c = index
    upper = (max(0, r - 1), c)
    lower = (min(puzzle.shape[1] - 1, r + 1), c)
    left = (r, max(0, c - 1))
    right = (r, min(puzzle.shape[0] - 1, c + 1))
    adjacent_coords = [upper, lower, left, right]
    
    adjacent_tiles = {puzzle[c]: tiles[puzzle[c]] for c in adjacent_coords if puzzle[c] != 0}
    n_adjacent = len(adjacent_tiles.keys())
    
    for k1, v1 in tiles.items():
        if k1 in puzzle or k1 in exclude_list:
            continue
        else:
            count = 0
            for k2, v2 in adjacent_tiles.items():
                if compare_tiles(tiles[k1], tiles[k2]):
                    count += 1
            if count == n_adjacent:
                return k1
    return None

In [6]:
def solve_puzzle(matching_edges):
    puzzle_size = int(np.sqrt(len(tiles.keys())))
    puzzle = np.zeros((puzzle_size, puzzle_size), dtype=int)
    puzzle[0, 0] = corner_ids[0]

    non_edge_tiles = [k for k, v in matching_edges.items() if v == 4]

    # Fill in the edges first
    for index, item in np.ndenumerate(puzzle):
        if item == 0 and (index[0] == 0 or index[1] == 0 or index[0] == puzzle_size - 1 or index[1] == puzzle_size - 1):
            puzzle[index] = find_piece(index, puzzle, non_edge_tiles)

    # Fill the inner pieces
    for index, item in np.ndenumerate(puzzle):
        if item == 0:
            puzzle[index] = find_piece(index, puzzle)
            
    return puzzle

In [7]:
puzzle = solve_puzzle(matching_edges)
puzzle

array([[3467, 1951, 1171, 1511, 1151, 3461, 1613, 1789, 2707, 2963, 1607,
        1249],
       [2719, 2731, 2549, 2273, 2237, 3209, 3767, 3733, 2477, 3923, 1697,
        2029],
       [3511, 3331, 2311, 3793, 3989, 1867, 1013, 1723, 3697, 1031, 2293,
        2129],
       [1129, 2837, 3253, 3847, 1933, 2339, 2027, 1163, 3739, 1873, 1091,
        3413],
       [1097, 3167, 2791, 1553, 3257, 1429, 2423, 3659, 2143, 3797, 2971,
        2411],
       [1549, 3251, 3547, 3469, 2579, 1879, 1721, 1931, 1489, 2887, 3391,
        1373],
       [1453, 1033, 1783, 1823, 3863, 2179, 2333, 3929, 3001, 2221, 3079,
        1801],
       [1871, 1523, 3931, 1997, 1069, 3643, 3163, 3907, 2393, 3583, 2251,
        1039],
       [2161, 3457, 3319, 1279, 2381, 1747, 2699, 1787, 3313, 1753, 2269,
        1229],
       [2711, 1213, 3853, 2879, 1223, 3299, 3671, 1949, 2797, 3917, 2287,
        2137],
       [1399, 3727, 2383, 2593, 2063, 1297, 3049, 3541, 2417, 1571, 1321,
        3761],
       [1019, 1283, 1

In [8]:
def transform(x, k):
    if k == 0:
        return x
    elif k == 1:
        return np.rot90(x)
    elif k == 2:
        return np.rot90(x, 2)
    elif k == 3:
        return np.rot90(x, 3)
    elif k == 4:
        return np.fliplr(x)
    elif k == 5:
        return np.fliplr(np.rot90(x))
    elif k == 6:
        return np.fliplr(np.rot90(x, 2))
    elif k == 7:
        return np.fliplr(np.rot90(x, 3))
    elif k == 8:
        return np.flipud(x)
    elif k == 9:
        return np.flipud(np.rot90(x))
    elif k == 10:
        return np.flipud(np.rot90(x, 2))
    elif k == 11:
        return np.flipud(np.rot90(x, 3))
    elif k == 12:
        return np.flip(x)
    elif k == 13:
        return np.flip(np.rot90(x))
    elif k == 14:
        return np.flip(np.rot90(x, 2))
    elif k == 15:
        return np.flip(np.rot90(x, 3))
    elif k == 16:
        return np.transpose(x)
    elif k == 17:
        return np.transpose(np.rot90(x))
    elif k == 18:
        return np.transpose(np.rot90(x, 2))
    elif k == 19:
        return np.transpose(np.rot90(x, 3))

In [9]:
def check_tile_alignment(a, b, c, d):
    cond1 = np.array_equal(a[:, -1], b[:, 0])
    cond2 = np.array_equal(c[:, -1], d[:, 0])
    cond3 = np.array_equal(a[-1], c[0])
    cond4 = np.array_equal(b[-1], d[0])
    return np.all([cond1, cond2, cond3, cond4])


# Brute force search in a 2x2 area [[a, b], [c, d]]
def align_tiles(tiles):
    a, b, c, d = tiles

    for i in range(20):
        for j in range(20):
            for k in range(20):
                for l in range(20):
                    a_t = transform(a, i)
                    b_t = transform(b, j)
                    c_t = transform(c, k)
                    d_t = transform(d, l)
                    if check_tile_alignment(a_t, b_t, c_t, d_t):
                        return a_t, b_t, c_t, d_t

In [10]:
# Orientate tiles
tiles_oriented = {}

for index, item in np.ndenumerate(puzzle[:-1, :-1]):
    row, col = index
    coords = [(row, col), (row, col + 1), (row + 1, col), (row + 1, col + 1)]
    tile_names = [puzzle[c] for c in coords]
    tiles2x2 = [tiles[n] for n in tile_names]

    out = align_tiles(tiles2x2)
    for name, img_part in zip(tile_names, out):
        if name not in tiles_oriented.keys():
            tiles_oriented[name] = img_part[1:-1, 1:-1]

# Build image
img = []
for row in puzzle:
    img.append(np.concatenate([tiles_oriented[i] for i in row], 1))
    
img = np.concatenate(img)

In [11]:
img.shape

(96, 96)

# Find the monsters
Brute force search (again!) with a sliding window

In [14]:
monster = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
           [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1],
           [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0]]
monster = np.array(monster)
monster.shape

(3, 20)

In [15]:
def check_for_monsters(img):
    count = 0
    for index, item in np.ndenumerate(img[:-monster.shape[0], :-monster.shape[1]]):
        row, col = index
        window = img[row:row + monster.shape[0], col:col+monster.shape[1]]

        if np.array_equal(window * monster, monster):
            count += 1
    if count > 0:
        print(f"Found {count} monsters")
        return count
    else:
        return None

In [16]:
for i in range(20):
    monster_count = check_for_monsters(transform(img, i))
    if monster_count is not None:
        print(np.sum(img) - monster_count * np.sum(monster))
        break

Found 34 monsters
2424
